<a href="https://colab.research.google.com/github/FinaFahrani/Kumpulan-Tugas_-Machine-Learning/blob/main/Final%20Exam/UAS_Fina_Fahrani_1103184098_Paper_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reproduce code : https://github.com/Vedurumudi-Priyanka/Twitter-Sentiment-Analysis


In [ ]:
!pip install twython # untuk API Twitter resmi

# **Utils**

In [2]:
import sys #mengakses konfigurasi interpreter
import pickle #untuk menyimpan dan membaca data ke dalam /dari sebuah file
import random #untuk  mengubah urutan setiap elemen yang ada pada list

#membuat class utils yang akan digunakan pada saat training dataset
def file_to_wordset(filename):#untuk converts file dari kata perbaris menjadi set python
    words = []
    with open(filename, 'r') as f:
        for line in f:
            words.append(line.strip())
    return set(words)


def write_status(i, total): #untuk melakukan penulisan status ke console
    sys.stdout.write('\r')
    sys.stdout.write('Processing %d/%d' % (i, total))
    sys.stdout.flush()


def save_results_to_csv(results, csv_file): #untuk menyimpan hasil dalam bentuk csv dalam format kaggle
    with open(csv_file, 'w') as csv:
        csv.write('id,prediction\n')
        for tweet_id, pred in results:
            csv.write(tweet_id)
            csv.write(',')
            csv.write(str(pred))
            csv.write('\n')


def top_n_words(pkl_file_name, N, shift=0): #untuk mengambil kata teratas dari file
    with open(pkl_file_name, 'rb') as pkl_file:
        freq_dist = pickle.load(pkl_file)
    most_common = freq_dist.most_common(N)
    words = {p[0]: i + shift for i, p in enumerate(most_common)}
    return words


def top_n_bigrams(pkl_file_name, N, shift=0): #untuk mengambil bigram teratas dari file yang memiliki objek Penghitung yang dihasilkan oleh stats.py
    with open(pkl_file_name, 'rb') as pkl_file:
        freq_dist = pickle.load(pkl_file)
    most_common = freq_dist.most_common(N)
    bigrams = {p[0]: i for i, p in enumerate(most_common)}
    return bigrams


def split_data(tweets, validation_split=0.1): #untuk split dataset menjadi training dan validation sets
    index = int((1 - validation_split) * len(tweets))
    random.shuffle(tweets)
    return tweets[:index], tweets[index:]


**Preprocessing**

In [ ]:
#mengambil alamat/path dataset train.csv lalu dilakukan preprocessed.py 
!python2 /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/code/preprocess.py /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train.csv

Processing 100000/100000
Saved processed tweets to: /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed.csv


In [ ]:
#mengambil alamat/path dataset test.csv lalu dilakukan preprocessed.py 
#pada file preproceseed.py line 107 bagian test_file diubah menjadi True
!python2 /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/code/preprocess.py /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/test.csv

Processing 300000/300000
Saved processed tweets to: /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/test-processed.csv


# **Stats**

In [ ]:
#mengecek dataset train-processed.csv yang telah di preprocessed untuk mengetahui berapa kata positive dan negative dalam dataset
!python2 /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/code/stats.py /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed.csv

Processing 100000/100000
Calculating frequency distribution
Saved uni-frequency distribution to /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist.pkl
Saved bi-frequency distribution to /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist-bi.pkl

[Analysis Statistics]
Tweets => Total: 100000, Positive: 56462, Negative: 43538
User Mentions => Total: 88959, Avg: 0.8896, Max: 12
URLs => Total: 3602, Avg: 0.0360, Max: 4
Emojis => Total: 1001, Positive: 856, Negative: 145, Avg: 0.0100, Max: 5
Words => Total: 1192424, Unique: 50376, Avg: 11.9242, Max: 40, Min: 0
Bigrams => Total: 1092958, Unique: 385024, Avg: 10.9296


# **Baseline**


In [ ]:
#klasifikasi tweet dari kata positif dan negatif menggunakan baseline

#dataset yang akan digunakan
TRAIN_PROCESSED_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed.csv'
TEST_PROCESSED_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/test-processed.csv'
POSITIVE_WORDS_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/positive-words.txt'
NEGATIVE_WORDS_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/positive-words.txt'
TRAIN = True


def classify(processed_csv, test_file=True, **params): #untuk melakukan klasifikasi data pada file yang sudah dilakukan preprocessed
    positive_words = file_to_wordset(params.pop('positive_words'))
    negative_words = file_to_wordset(params.pop('negative_words'))
    predictions = []
    with open(processed_csv, 'r') as csv: #membuka file preprocessed sebagai csv
        for line in csv:
            if test_file:
                tweet_id, tweet = line.strip().split(',')
            else:
                tweet_id, label, tweet = line.strip().split(',')
            pos_count, neg_count = 0, 0
            for word in tweet.split():
                if word in positive_words:
                    pos_count += 1
                elif word in negative_words:
                    neg_count += 1
            #print (pos_count, neg_count)
            prediction = 1 if pos_count >= neg_count else 0 #untuk melakukan prediksi dataset 
            if test_file:
                predictions.append((tweet_id, prediction))
            else:
                predictions.append((tweet_id, int(label), prediction))
    return predictions


if __name__ == '__main__': #untuk menampilkan dan menyimpan hasil prediksi dataset 
    if TRAIN:
        predictions = classify(TRAIN_PROCESSED_FILE, test_file=(not TRAIN), positive_words=POSITIVE_WORDS_FILE, negative_words=NEGATIVE_WORDS_FILE)
        correct = sum([1 for p in predictions if p[1] == p[2]]) * 100.0 / len(predictions)
        #menampilkan hasil akurasi training dataset
        print ('Correct = %.2f%%' % correct)
    else:
        predictions = classify(TEST_PROCESSED_FILE, test_file=(not TRAIN), positive_words=POSITIVE_WORDS_FILE, negative_words=NEGATIVE_WORDS_FILE)
        save_results_to_csv(predictions, '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/baseline.csv')


Correct = 56.46%


# **Naive Bayes**

In [ ]:
import random #untuk  mengubah urutan setiap elemen yang ada pada list
import numpy as np #untuk melakukan komputasi numerik
from sklearn.naive_bayes import MultinomialNB #untuk klasifikasi dengan fitur diskrit (misalnya, jumlah kata untuk klasifikasi teks).
from scipy.sparse import lil_matrix #menangani sparse data dan  membuat matriks
from sklearn.feature_extraction.text import TfidfTransformer #untuk pembobotan kata dalam klasifikasi dokumen.
import pandas as pd #untuk  mengecek data
# Performs classification using Naive Bayes.

#dataset dan variabel yang dibutuhkan
FREQ_DIST_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist.pkl'
BI_FREQ_DIST_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist-bi.pkl'
TRAIN_PROCESSED_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed.csv'
TEST_PROCESSED_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/test-processed.csv'
TRAIN = True
UNIGRAM_SIZE = 15000
VOCAB_SIZE = UNIGRAM_SIZE
USE_BIGRAMS = True
if USE_BIGRAMS:
    BIGRAM_SIZE = 10000
    VOCAB_SIZE = UNIGRAM_SIZE + BIGRAM_SIZE
FEAT_TYPE = 'frequency'
xrange = range

def get_feature_vector(tweet): #untuk mendapatkan vektor pada tweet
    uni_feature_vector = []
    bi_feature_vector = []
    words = tweet.split()
    for i in xrange(len(words) - 1):
        word = words[i]
        next_word = words[i + 1]
        if unigrams.get(word):
            uni_feature_vector.append(word)
        if USE_BIGRAMS:
            if bigrams.get((word, next_word)):
                bi_feature_vector.append((word, next_word))
    if len(words) >= 1:
        if unigrams.get(words[-1]):
            uni_feature_vector.append(words[-1])
    return uni_feature_vector, bi_feature_vector


def extract_features(tweets, batch_size=500, test_file=True, feat_type='presence'): #melakukan ekstrasi feature tweet
    num_batches = int(np.ceil(len(tweets) / float(batch_size)))
    for i in xrange(num_batches):
        batch = tweets[i * batch_size: (i + 1) * batch_size]
        features = lil_matrix((batch_size, VOCAB_SIZE))
        labels = np.zeros(batch_size)
        for j, tweet in enumerate(batch):
            if test_file:
                tweet_words = tweet[1][0]
                tweet_bigrams = tweet[1][1]
            else:
                tweet_words = tweet[2][0]
                tweet_bigrams = tweet[2][1]
                labels[j] = tweet[1]
            if feat_type == 'presence':
                tweet_words = set(tweet_words)
                tweet_bigrams = set(tweet_bigrams)
            for word in tweet_words:
                idx = unigrams.get(word)
                if idx:
                    features[j, idx] += 1
            if USE_BIGRAMS:
                for bigram in tweet_bigrams:
                    idx = bigrams.get(bigram)
                    if idx:
                        features[j, UNIGRAM_SIZE + idx] += 1
        yield features, labels


def apply_tf_idf(X): #untuk mendapatkan nilai tf idf 
    transformer = TfidfTransformer(smooth_idf=True, sublinear_tf=True, use_idf=True)
    transformer.fit(X)
    return transformer


def process_tweets(csv_file, test_file=True): #untuk mendapatkan vector feature
    """Returns a list of tuples of type (tweet_id, feature_vector)
            or (tweet_id, sentiment, feature_vector)
    Args:
        csv_file (str): Name of processed csv file generated by preprocess.py
        test_file (bool, optional): If processing test file
    Returns:
        list: Of tuples
    """
    tweets = []
    print ('Generating feature vectors')
    with open(csv_file, 'r') as csv:
        lines = csv.readlines()
        total = len(lines)
        for i, line in enumerate(lines):
            if test_file:
                tweet_id, tweet = line.split(',')
            else:
                tweet_id, sentiment, tweet = line.split(',')
            feature_vector = get_feature_vector(tweet)
            if test_file:
                tweets.append((tweet_id, feature_vector))
            else:
                tweets.append((tweet_id, int(sentiment), feature_vector))
            write_status(i + 1, total)
    print ('\n')
    return tweets


if __name__ == '__main__':
    np.random.seed(1337)
    unigrams = top_n_words(FREQ_DIST_FILE, UNIGRAM_SIZE)
    if USE_BIGRAMS:
        bigrams = top_n_bigrams(BI_FREQ_DIST_FILE, BIGRAM_SIZE)
    tweets = process_tweets(TRAIN_PROCESSED_FILE, test_file=False)
    if TRAIN:
        train_tweets, val_tweets = split_data(tweets)
    else:
        random.shuffle(tweets)
        train_tweets = tweets
    del tweets
    print ('Extracting features & training batches')
    clf = MultinomialNB()
    batch_size = len(train_tweets)
    i = 1
    n_train_batches = int(np.ceil(len(train_tweets) / float(batch_size)))
    for training_set_X, training_set_y in extract_features(train_tweets, test_file=False, feat_type=FEAT_TYPE, batch_size=batch_size):
        write_status(i, n_train_batches)
        i += 1
        if FEAT_TYPE == 'frequency':
            tfidf = apply_tf_idf(training_set_X)
            training_set_X = tfidf.transform(training_set_X)
        clf.partial_fit(training_set_X, training_set_y, classes=[0, 1])
    print ('\n')
    print ('Testing')
    if TRAIN:
        correct, total = 0, len(val_tweets)
        i = 1
        batch_size = len(val_tweets)
        n_val_batches = int(np.ceil(len(val_tweets) / float(batch_size)))
        for val_set_X, val_set_y in extract_features(val_tweets, test_file=False, feat_type=FEAT_TYPE, batch_size=batch_size):
            if FEAT_TYPE == 'frequency': #melakukan prediksi dataset
                val_set_X = tfidf.transform(val_set_X)
            prediction = clf.predict(val_set_X)
            correct += np.sum(prediction == val_set_y)
            write_status(i, n_val_batches)
            i += 1
        print ('\nCorrect: %d/%d = %.4f %%' % (correct, total, correct * 100. / total))
    else:
        del train_tweets
        test_tweets = process_tweets(TEST_PROCESSED_FILE, test_file=True)
        n_test_batches = int(np.ceil(len(test_tweets) / float(batch_size)))
        predictions = np.array([])
        print ('Predicting batches')
        i = 1
        for test_set_X, _ in extract_features(test_tweets, test_file=True, feat_type=FEAT_TYPE):
            if FEAT_TYPE == 'frequency':
                test_set_X = tfidf.transform(test_set_X)
            prediction = clf.predict(test_set_X)
            predictions = np.concatenate((predictions, prediction))
            write_status(i, n_test_batches)
            i += 1
        predictions = [(str(j), int(predictions[j]))
                       for j in range(len(test_tweets))]
        save_results_to_csv(predictions, '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/naivebayes.csv')
        print ('\nSaved to naivebayes.csv')


Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1

Testing
Processing 1/1
Correct: 7791/10000 = 77.9100 %


In [ ]:
!python2 /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/code/naivebayes.py

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1

Testing
Processing 1/1
Correct: 7777/10000 = 77.7700 %


# **Maximum Entropy**

maxent-nltk.py


In [ ]:
import nltk #set library untuk NLP
import random  #untuk  mengubah urutan setiap elemen yang ada pada list
import pickle #untuk menyimpan dan membaca data ke dalam /dari sebuah file
import sys #mengakses konfigurasi interpreter
import numpy as np #untuk melakukan komputasi numerik
# Performs classification using Maximun Entropy.

#dataset serta atribut yang diperlukan
TRAIN_PROCESSED_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed.csv'
TEST_PROCESSED_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/test-processed.csv'
USE_BIGRAMS = False
TRAIN = True
xrange = range

def get_data_from_file(file_name, isTrain=True): #untuk mendapatkan data dari file yang akan di klasifikasi
    data = []
    with open(train_csv_file, 'r') as csv:
        lines = csv.readlines()
        total = len(lines)
        for i, line in enumerate(lines):
            if isTrain:
                tag = line.split(',')[1]
                bag_of_words = line.split(',')[2].split()
                if USE_BIGRAMS:
                    bag_of_words_bigram = list(nltk.bigrams(line.split(',')[2].split()))
                    bag_of_words = bag_of_words+bag_of_words_bigram
            else :
                tag = '5'
                bag_of_words = line.split(',')[1].split()
                if USE_BIGRAMS:
                    bag_of_words_bigram = list(nltk.bigrams(line.split(',')[1].split()))
                    bag_of_words = bag_of_words+bag_of_words_bigram
            data.append((bag_of_words, tag))
    return data

def split_data(tweets, validation_split=0.1): #untuk validation/test split dataset
    index = int((1 - validation_split) * len(tweets))
    random.shuffle(tweets)
    return tweets[:index], tweets[index:]

def list_to_dict(words_list): #untuk membuat word list
    return dict([(word, True) for word in words_list])

if __name__ == '__main__':
    train = True
    np.random.seed(1337)
    train_csv_file = TRAIN_PROCESSED_FILE
    test_csv_file = TEST_PROCESSED_FILE
    train_data = get_data_from_file(train_csv_file, isTrain=True)
    train_set, validation_set = split_data(train_data)
    training_set_formatted = [(list_to_dict(element[0]), element[1]) for element in train_set]
    validation_set_formatted = [(list_to_dict(element[0]), element[1]) for element in validation_set]
    numIterations = 1
    algorithm = nltk.classify.MaxentClassifier.ALGORITHMS[1]
    classifier = nltk.MaxentClassifier.train(training_set_formatted, algorithm, max_iter=numIterations)
    classifier.show_most_informative_features(10)
    count = int(0)
    for review in validation_set_formatted: #untuk melakukasi prediksi akurasi
        label = review[1]
        text = review[0]
        determined_label = classifier.classify(text)
        #print(determined_label, label)
        if determined_label!=label:
            count+=int(1)
    accuracy = (len(validation_set)-count)/len(validation_set)
    print ('Validation set accuracy:%.4f'% (accuracy))
    f = open('maxEnt_classifier.pickle', 'wb')
    pickle.dump(classifier, f)
    f.close()
    print ('\nPredicting for test data')
    test_data = get_data_from_file(test_csv_file, isTrain=False)
    test_set_formatted = [(list_to_dict(element[0]), element[1]) for element in test_data]
    tweet_id = int(0)
    results = []
    for review in test_set_formatted:
        text = review[0]
        label = classifier.classify(text)
        results.append((str(tweet_id), label))
        tweet_id += int(1)
    save_results_to_csv(results, '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/maxent.csv')
    print ('\nSaved to maxent.csv')

  ==> Training (1 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.565
         Final          -0.61640        0.756
  -1.010 andyhurleyday==True and label is '0'
   1.000 atthepool==True and label is '1'
   1.000 princecharming==True and label is '0'
   1.000 owiree==True and label is '0'
   1.000 halfbloodprince==True and label is '1'
   1.000 harpersglobe==True and label is '1'
   1.000 hiccuuppss==True and label is '0'
   1.000 thestreetforce==True and label is '1'
   1.000 madnessness==True and label is '0'
   1.000 hitech==True and label is '1'
Validation set accuracy:0.7062

Predicting for test data

Saved to maxent.csv


# **Decision Tree**

In [ ]:
import random #untuk  mengubah urutan setiap elemen yang ada pada list
import numpy as np #untuk melakukan komputasi numerik
from sklearn.tree import DecisionTreeClassifier #untuk menggunakan DecisionTreeClassifier dari sklearn
from scipy.sparse import lil_matrix #menangani sparse data dan  membuat matriks
from sklearn.feature_extraction.text import TfidfTransformer

# Performs classification using Decision Tree.
#dataset untuk klasifikasi decision tree
FREQ_DIST_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist.pkl'
BI_FREQ_DIST_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist-bi.pkl'
TRAIN_PROCESSED_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed.csv'
TEST_PROCESSED_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/test-processed.csv'

# True while training.
TRAIN = True
UNIGRAM_SIZE = 15000
VOCAB_SIZE = UNIGRAM_SIZE

# If using bigrams.
USE_BIGRAMS = False
if USE_BIGRAMS:
    BIGRAM_SIZE = 10000
    VOCAB_SIZE = UNIGRAM_SIZE + BIGRAM_SIZE
FEAT_TYPE = 'frequency'
xrange = range

def get_feature_vector(tweet): #untuk mendapatkan vektor pada tweet
    uni_feature_vector = []
    bi_feature_vector = []
    words = tweet.split()
    for i in xrange(len(words) - 1):
        word = words[i]
        next_word = words[i + 1]
        if unigrams.get(word):
            uni_feature_vector.append(word)
        if USE_BIGRAMS:
            if bigrams.get((word, next_word)):
                bi_feature_vector.append((word, next_word))
    if len(words) >= 1:
        if unigrams.get(words[-1]):
            uni_feature_vector.append(words[-1])
    return uni_feature_vector, bi_feature_vector


def extract_features(tweets, batch_size=500, test_file=True, feat_type='presence'): #melakukan ekstrasi feature tweet
    num_batches = int(np.ceil(len(tweets) / float(batch_size)))
    for i in xrange(num_batches):
        batch = tweets[i * batch_size: (i + 1) * batch_size]
        features = lil_matrix((batch_size, VOCAB_SIZE))
        labels = np.zeros(batch_size)
        for j, tweet in enumerate(batch):
            if test_file:
                tweet_words = tweet[1][0]
                tweet_bigrams = tweet[1][1]
            else:
                tweet_words = tweet[2][0]
                tweet_bigrams = tweet[2][1]
                labels[j] = tweet[1]
            if feat_type == 'presence':
                tweet_words = set(tweet_words)
                tweet_bigrams = set(tweet_bigrams)
            for word in tweet_words:
                idx = unigrams.get(word)
                if idx:
                    features[j, idx] += 1
            if USE_BIGRAMS:
                for bigram in tweet_bigrams:
                    idx = bigrams.get(bigram)
                    if idx:
                        features[j, UNIGRAM_SIZE + idx] += 1
        yield features, labels


def apply_tf_idf(X):
    """
    Fits X for TF-IDF vectorization and returns the transformer.
    """
    transformer = TfidfTransformer(smooth_idf=True, sublinear_tf=True, use_idf=True)
    transformer.fit(X)
    return transformer


def process_tweets(csv_file, test_file=True):
    """Returns a list of tuples of type (tweet_id, feature_vector)
            or (tweet_id, sentiment, feature_vector)
    Args:
        csv_file (str): Name of processed csv file generated by preprocess.py
        test_file (bool, optional): If processing test file
    Returns:
        list: Of tuples
    """
    tweets = []
    print ('Generating feature vectors')
    with open(csv_file, 'r') as csv:
        lines = csv.readlines()
        total = len(lines)
        for i, line in enumerate(lines):
            if test_file:
                tweet_id, tweet = line.split(',')
            else:
                tweet_id, sentiment, tweet = line.split(',')
            feature_vector = get_feature_vector(tweet)
            if test_file:
                tweets.append((tweet_id, feature_vector))
            else:
                tweets.append((tweet_id, int(sentiment), feature_vector))
            write_status(i + 1, total)
    print ('\n')
    return tweets


if __name__ == '__main__':
    np.random.seed(1337)
    unigrams =top_n_words(FREQ_DIST_FILE, UNIGRAM_SIZE)
    if USE_BIGRAMS:
        bigrams = top_n_bigrams(BI_FREQ_DIST_FILE, BIGRAM_SIZE)
    tweets = process_tweets(TRAIN_PROCESSED_FILE, test_file=False)
    if TRAIN:
        train_tweets, val_tweets = split_data(tweets)
    else:
        random.shuffle(tweets)
        train_tweets = tweets
    del tweets
    print ('Extracting features & training batches')
    clf = DecisionTreeClassifier(max_depth=25)
    batch_size = len(train_tweets)
    i = 1
    n_train_batches = int(np.ceil(len(train_tweets) / float(batch_size)))
    for training_set_X, training_set_y in extract_features(train_tweets, test_file=False, feat_type=FEAT_TYPE, batch_size=batch_size):
        write_status(i, n_train_batches)
        i += 1
        if FEAT_TYPE == 'frequency':
            tfidf = apply_tf_idf(training_set_X)
            training_set_X = tfidf.transform(training_set_X)
        clf.fit(training_set_X, training_set_y)
    print ('\n')
    print ('Testing')
    if TRAIN:
        correct, total = 0, len(val_tweets)
        i = 1
        batch_size = len(val_tweets)
        n_val_batches = int(np.ceil(len(val_tweets) / float(batch_size)))
        for val_set_X, val_set_y in extract_features(val_tweets, test_file=False, feat_type=FEAT_TYPE, batch_size=batch_size):
            if FEAT_TYPE == 'frequency':
                val_set_X = tfidf.transform(val_set_X)
            prediction = clf.predict(val_set_X)
            correct += np.sum(prediction == val_set_y)
            write_status(i, n_val_batches)
            i += 1
        print ('\nCorrect: %d/%d = %.4f %%' % (correct, total, correct * 100. / total))
    else:
        del train_tweets
        test_tweets = process_tweets(TEST_PROCESSED_FILE, test_file=True)
        n_test_batches = int(np.ceil(len(test_tweets) / float(batch_size)))
        predictions = np.array([])
        print ('Predicting batches')
        i = 1
        for test_set_X, _ in extract_features(test_tweets, test_file=True, feat_type=FEAT_TYPE):
            if FEAT_TYPE == 'frequency':
                test_set_X = tfidf.transform(test_set_X)
            prediction = clf.predict(test_set_X)
            predictions = np.concatenate((predictions, prediction))
            write_status(i, n_test_batches)
            i += 1
        predictions = [(str(j), int(predictions[j]))
                       for j in range(len(test_tweets))]
        save_results_to_csv(predictions, '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/decisiontree.csv')
        print ('\nSaved to decisiontree.csv')

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1

Testing
Processing 1/1
Correct: 6853/10000 = 68.5300 %


In [ ]:
!python2 /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/code/decisiontree.py

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1

Testing
Processing 1/1
Correct: 6814/10000 = 68.1400 %


# **Random Forest**

In [ ]:
import random #untuk  mengubah urutan setiap elemen yang ada pada list
import numpy as np #untuk melakukan komputasi numerik
from sklearn.ensemble import RandomForestClassifier #untuk menggunakan  metode random forest classifier 
from scipy.sparse import lil_matrix #menangani sparse data dan  membuat matriks
from sklearn.feature_extraction.text import TfidfTransformer #untuk pembobotan kata dalam klasifikasi dokumen.

# Performs classification using RandomForest classifier.
#dataset dan variabel yang dibutuhkan
FREQ_DIST_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist.pkl'
BI_FREQ_DIST_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist-bi.pkl'
TRAIN_PROCESSED_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed.csv'
TEST_PROCESSED_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/test-processed.csv'
TRAIN = True
UNIGRAM_SIZE = 15000
VOCAB_SIZE = UNIGRAM_SIZE
USE_BIGRAMS = False
if USE_BIGRAMS:
    BIGRAM_SIZE = 10000
    VOCAB_SIZE = UNIGRAM_SIZE + BIGRAM_SIZE
FEAT_TYPE = 'presence'
xrange = range

def get_feature_vector(tweet): #untuk mendapatkan vektor pada tweet
    uni_feature_vector = []
    bi_feature_vector = []
    words = tweet.split()
    for i in xrange(len(words) - 1):
        word = words[i]
        next_word = words[i + 1]
        if unigrams.get(word):
            uni_feature_vector.append(word)
        if USE_BIGRAMS:
            if bigrams.get((word, next_word)):
                bi_feature_vector.append((word, next_word))
    if len(words) >= 1:
        if unigrams.get(words[-1]):
            uni_feature_vector.append(words[-1])
    return uni_feature_vector, bi_feature_vector


def extract_features(tweets, batch_size=500, test_file=True, feat_type='presence'): #melakukan ekstrasi feature tweet
    num_batches = int(np.ceil(len(tweets) / float(batch_size)))
    for i in xrange(num_batches):
        batch = tweets[i * batch_size: (i + 1) * batch_size]
        features = lil_matrix((batch_size, VOCAB_SIZE))
        labels = np.zeros(batch_size)
        for j, tweet in enumerate(batch):
            if test_file:
                tweet_words = tweet[1][0]
                tweet_bigrams = tweet[1][1]
            else:
                tweet_words = tweet[2][0]
                tweet_bigrams = tweet[2][1]
                labels[j] = tweet[1]
            if feat_type == 'presence':
                tweet_words = set(tweet_words)
                tweet_bigrams = set(tweet_bigrams)
            for word in tweet_words:
                idx = unigrams.get(word)
                if idx:
                    features[j, idx] += 1
            if USE_BIGRAMS:
                for bigram in tweet_bigrams:
                    idx = bigrams.get(bigram)
                    if idx:
                        features[j, UNIGRAM_SIZE + idx] += 1
        yield features, labels


def apply_tf_idf(X):
    transformer = TfidfTransformer(smooth_idf=True, sublinear_tf=True, use_idf=True)
    transformer.fit(X)
    return transformer


def process_tweets(csv_file, test_file=True):
    """Returns a list of tuples of type (tweet_id, feature_vector)
            or (tweet_id, sentiment, feature_vector)
    Args:
        csv_file (str): Name of processed csv file generated by preprocess.py
        test_file (bool, optional): If processing test file
    Returns:
        list: Of tuples
    """
    tweets = []
    print ('Generating feature vectors')
    with open(csv_file, 'r') as csv:
        lines = csv.readlines()
        total = len(lines)
        for i, line in enumerate(lines):
            if test_file:
                tweet_id, tweet = line.split(',')
            else:
                tweet_id, sentiment, tweet = line.split(',')
            feature_vector = get_feature_vector(tweet)
            if test_file:
                tweets.append((tweet_id, feature_vector))
            else:
                tweets.append((tweet_id, int(sentiment), feature_vector))
            write_status(i + 1, total)
    print ('\n')
    return tweets


if __name__ == '__main__':
    np.random.seed(1337)
    unigrams = top_n_words(FREQ_DIST_FILE, UNIGRAM_SIZE)
    if USE_BIGRAMS:
        bigrams = top_n_bigrams(BI_FREQ_DIST_FILE, BIGRAM_SIZE)
    tweets = process_tweets(TRAIN_PROCESSED_FILE, test_file=False)
    if TRAIN:
        train_tweets, val_tweets = split_data(tweets)
    else:
        random.shuffle(tweets)
        train_tweets = tweets
    del tweets
    print ('Extracting features & training batches')
    clf = RandomForestClassifier(n_jobs=2, random_state=0)
    batch_size = len(train_tweets)
    i = 1
    n_train_batches = int(np.ceil(len(train_tweets) / float(batch_size)))
    for training_set_X, training_set_y in extract_features(train_tweets, test_file=False, feat_type=FEAT_TYPE, batch_size=batch_size):
        write_status(i, n_train_batches)
        i += 1
        if FEAT_TYPE == 'frequency':
            tfidf = apply_tf_idf(training_set_X)
            training_set_X = tfidf.transform(training_set_X)
        clf.fit(training_set_X, training_set_y)
    print ('\n')
    print ('Testing')
    if TRAIN:
        correct, total = 0, len(val_tweets)
        i = 1
        batch_size = len(val_tweets)
        n_val_batches = int(np.ceil(len(val_tweets) / float(batch_size)))
        for val_set_X, val_set_y in extract_features(val_tweets, test_file=False, feat_type=FEAT_TYPE, batch_size=batch_size):
            if FEAT_TYPE == 'frequency':
                val_set_X = tfidf.transform(val_set_X)
            prediction = clf.predict(val_set_X)
            correct += np.sum(prediction == val_set_y)
            write_status(i, n_val_batches)
            i += 1
        print ('\nCorrect: %d/%d = %.4f %%' % (correct, total, correct * 100. / total))
    else:
        del train_tweets
        test_tweets = process_tweets(TEST_PROCESSED_FILE, test_file=True)
        n_test_batches = int(np.ceil(len(test_tweets) / float(batch_size)))
        predictions = np.array([])
        print ('Predicting batches')
        i = 1
        for test_set_X, _ in extract_features(test_tweets, test_file=True, feat_type=FEAT_TYPE):
            if FEAT_TYPE == 'frequency':
                test_set_X = tfidf.transform(test_set_X)
            prediction = clf.predict(test_set_X)
            predictions = np.concatenate((predictions, prediction))
            write_status(i, n_test_batches)
            i += 1
        predictions = [(str(j), int(predictions[j]))
                       for j in range(len(test_tweets))]
        save_results_to_csv(predictions, '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/randomforest.csv')
        print ('\nSaved to randomforest.csv')

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1

Testing
Processing 1/1
Correct: 7582/10000 = 75.8200 %


In [ ]:
!python2 /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/code/randomforest.py

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1/usr/local/lib/python2.7/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Testing
Processing 1/1
Correct: 7174/10000 = 71.7400 %


# **XGBoost**

In [ ]:
import random #untuk  mengubah urutan setiap elemen yang ada pada list
import numpy as np #untuk melakukan komputasi numerik
from xgboost import XGBClassifier
from scipy.sparse import lil_matrix #menangani sparse data dan  membuat matriks
from sklearn.feature_extraction.text import TfidfTransformer #untuk pembobotan kata dalam klasifikasi dokumen.

# Performs classification using XGBoost.


FREQ_DIST_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist.pkl'
BI_FREQ_DIST_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist-bi.pkl'
TRAIN_PROCESSED_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed.csv'
TEST_PROCESSED_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/test-processed.csv'
TRAIN = True
UNIGRAM_SIZE = 15000
VOCAB_SIZE = UNIGRAM_SIZE
USE_BIGRAMS = True
if USE_BIGRAMS:
    BIGRAM_SIZE = 10000
    VOCAB_SIZE = UNIGRAM_SIZE + BIGRAM_SIZE
FEAT_TYPE = 'frequency'
xrange = range

def get_feature_vector(tweet): #untuk mendapatkan vektor pada tweet
    uni_feature_vector = []
    bi_feature_vector = []
    words = tweet.split()
    for i in xrange(len(words) - 1):
        word = words[i]
        next_word = words[i + 1]
        if unigrams.get(word):
            uni_feature_vector.append(word)
        if USE_BIGRAMS:
            if bigrams.get((word, next_word)):
                bi_feature_vector.append((word, next_word))
    if len(words) >= 1:
        if unigrams.get(words[-1]):
            uni_feature_vector.append(words[-1])
    return uni_feature_vector, bi_feature_vector


def extract_features(tweets, batch_size=500, test_file=True, feat_type='presence'): #melakukan ekstrasi feature tweet
    num_batches = int(np.ceil(len(tweets) / float(batch_size)))
    for i in xrange(num_batches):
        batch = tweets[i * batch_size: (i + 1) * batch_size]
        features = lil_matrix((batch_size, VOCAB_SIZE))
        labels = np.zeros(batch_size)
        for j, tweet in enumerate(batch):
            if test_file:
                tweet_words = tweet[1][0]
                tweet_bigrams = tweet[1][1]
            else:
                tweet_words = tweet[2][0]
                tweet_bigrams = tweet[2][1]
                labels[j] = tweet[1]
            if feat_type == 'presence':
                tweet_words = set(tweet_words)
                tweet_bigrams = set(tweet_bigrams)
            for word in tweet_words:
                idx = unigrams.get(word)
                if idx:
                    features[j, idx] += 1
            if USE_BIGRAMS:
                for bigram in tweet_bigrams:
                    idx = bigrams.get(bigram)
                    if idx:
                        features[j, UNIGRAM_SIZE + idx] += 1
        yield features, labels


def apply_tf_idf(X):
    transformer = TfidfTransformer(smooth_idf=True, sublinear_tf=True, use_idf=True)
    transformer.fit(X)
    return transformer


def process_tweets(csv_file, test_file=True):
    """Returns a list of tuples of type (tweet_id, feature_vector)
            or (tweet_id, sentiment, feature_vector)
    Args:
        csv_file (str): Name of processed csv file generated by preprocess.py
        test_file (bool, optional): If processing test file
    Returns:
        list: Of tuples
    """
    tweets = []
    print ('Generating feature vectors')
    with open(csv_file, 'r') as csv:
        lines = csv.readlines()
        total = len(lines)
        for i, line in enumerate(lines):
            if test_file:
                tweet_id, tweet = line.split(',')
            else:
                tweet_id, sentiment, tweet = line.split(',')
            feature_vector = get_feature_vector(tweet)
            if test_file:
                tweets.append((tweet_id, feature_vector))
            else:
                tweets.append((tweet_id, int(sentiment), feature_vector))
            write_status(i + 1, total)
    print ('\n')
    return tweets


if __name__ == '__main__':
    np.random.seed(1337)
    unigrams = top_n_words(FREQ_DIST_FILE, UNIGRAM_SIZE)
    if USE_BIGRAMS:
        bigrams = top_n_bigrams(BI_FREQ_DIST_FILE, BIGRAM_SIZE)
    tweets = process_tweets(TRAIN_PROCESSED_FILE, test_file=False)
    if TRAIN:
        train_tweets, val_tweets =split_data(tweets)
    else:
        random.shuffle(tweets)
        train_tweets = tweets
    del tweets
    print ('Extracting features & training batches')
    clf = XGBClassifier(max_depth=25, silent=False, n_estimators=400)
    batch_size = len(train_tweets)
    i = 1
    n_train_batches = int(np.ceil(len(train_tweets) / float(batch_size)))
    for training_set_X, training_set_y in extract_features(train_tweets, test_file=False, feat_type=FEAT_TYPE, batch_size=batch_size):
        write_status(i, n_train_batches)
        i += 1
        if FEAT_TYPE == 'frequency':
            tfidf = apply_tf_idf(training_set_X)
            training_set_X = tfidf.transform(training_set_X)
        clf.fit(training_set_X, training_set_y)
    print ('\n')
    print ('Testing')
    if TRAIN:
        correct, total = 0, len(val_tweets)
        i = 1
        batch_size = len(val_tweets)
        n_val_batches = int(np.ceil(len(val_tweets) / float(batch_size)))
        for val_set_X, val_set_y in extract_features(val_tweets, test_file=False, feat_type=FEAT_TYPE, batch_size=batch_size):
            if FEAT_TYPE == 'frequency':
                val_set_X = tfidf.transform(val_set_X)
            prediction = clf.predict(val_set_X)
            correct += np.sum(prediction == val_set_y)
            write_status(i, n_val_batches)
            i += 1
        print ('\nCorrect: %d/%d = %.4f %%' % (correct, total, correct * 100. / total))
    else:
        del train_tweets
        test_tweets = process_tweets(TEST_PROCESSED_FILE, test_file=True)
        n_test_batches = int(np.ceil(len(test_tweets) / float(batch_size)))
        predictions = np.array([])
        print ('Predicting batches')
        i = 1
        for test_set_X, _ in extract_features(test_tweets, test_file=True, feat_type=FEAT_TYPE):
            if FEAT_TYPE == 'frequency':
                test_set_X = tfidf.transform(test_set_X)
            prediction = clf.predict(test_set_X)
            predictions = np.concatenate((predictions, prediction))
            write_status(i, n_test_batches)
            i += 1
        predictions = [(str(j), int(predictions[j]))
                       for j in range(len(test_tweets))]
        save_results_to_csv(predictions, '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/xgboost.csv')
        print ('\nSaved to xgboost.csv')

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1

Testing
Processing 1/1
Correct: 7630/10000 = 76.3000 %


# **SVM**

In [ ]:
from sklearn import svm #untuk menggukan klasifikasi SVM
import random #untuk  mengubah urutan setiap elemen yang ada pada list
import numpy as np #untuk melakukan komputasi numerik
from scipy.sparse import lil_matrix #menangani sparse data dan  membuat matriks
from sklearn.feature_extraction.text import TfidfTransformer #untuk pembobotan kata dalam klasifikasi dokumen.

# Performs classification using SVM.

FREQ_DIST_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist.pkl'
BI_FREQ_DIST_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist-bi.pkl'
TRAIN_PROCESSED_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed.csv'
TEST_PROCESSED_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/test-processed.csv'
TRAIN = True
UNIGRAM_SIZE = 15000
VOCAB_SIZE = UNIGRAM_SIZE
USE_BIGRAMS = True
if USE_BIGRAMS:
    BIGRAM_SIZE = 10000
    VOCAB_SIZE = UNIGRAM_SIZE + BIGRAM_SIZE
FEAT_TYPE = 'frequency'
xrange = range

def get_feature_vector(tweet): #untuk mendapatkan vektor pada tweet
    uni_feature_vector = []
    bi_feature_vector = []
    words = tweet.split()
    for i in xrange(len(words) - 1):
        word = words[i]
        next_word = words[i + 1]
        if unigrams.get(word):
            uni_feature_vector.append(word)
        if USE_BIGRAMS:
            if bigrams.get((word, next_word)):
                bi_feature_vector.append((word, next_word))
    if len(words) >= 1:
        if unigrams.get(words[-1]):
            uni_feature_vector.append(words[-1])
    return uni_feature_vector, bi_feature_vector


def extract_features(tweets, batch_size=500, test_file=True, feat_type='presence'): #melakukan ekstrasi feature tweet
    num_batches = int(np.ceil(len(tweets) / float(batch_size)))
    for i in xrange(num_batches):
        batch = tweets[i * batch_size: (i + 1) * batch_size]
        features = lil_matrix((batch_size, VOCAB_SIZE))
        labels = np.zeros(batch_size)
        for j, tweet in enumerate(batch):
            if test_file:
                tweet_words = tweet[1][0]
                tweet_bigrams = tweet[1][1]
            else:
                tweet_words = tweet[2][0]
                tweet_bigrams = tweet[2][1]
                labels[j] = tweet[1]
            if feat_type == 'presence':
                tweet_words = set(tweet_words)
                tweet_bigrams = set(tweet_bigrams)
            for word in tweet_words:
                idx = unigrams.get(word)
                if idx:
                    features[j, idx] += 1
            if USE_BIGRAMS:
                for bigram in tweet_bigrams:
                    idx = bigrams.get(bigram)
                    if idx:
                        features[j, UNIGRAM_SIZE + idx] += 1
        yield features, labels


def apply_tf_idf(X):
    transformer = TfidfTransformer(smooth_idf=True, sublinear_tf=True, use_idf=True)
    transformer.fit(X)
    return transformer


def process_tweets(csv_file, test_file=True):
    """Returns a list of tuples of type (tweet_id, feature_vector)
            or (tweet_id, sentiment, feature_vector)
    Args:
        csv_file (str): Name of processed csv file generated by preprocess.py
        test_file (bool, optional): If processing test file
    Returns:
        list: Of tuples
    """
    tweets = []
    print ('Generating feature vectors')
    with open(csv_file, 'r') as csv:
        lines = csv.readlines()
        total = len(lines)
        for i, line in enumerate(lines):
            if test_file:
                tweet_id, tweet = line.split(',')
            else:
                tweet_id, sentiment, tweet = line.split(',')
            feature_vector = get_feature_vector(tweet)
            if test_file:
                tweets.append((tweet_id, feature_vector))
            else:
                tweets.append((tweet_id, int(sentiment), feature_vector))
            write_status(i + 1, total)
    print ('\n')
    return tweets


if __name__ == '__main__':
    np.random.seed(1337)
    unigrams = top_n_words(FREQ_DIST_FILE, UNIGRAM_SIZE)
    if USE_BIGRAMS:
        bigrams = top_n_bigrams(BI_FREQ_DIST_FILE, BIGRAM_SIZE)
    tweets = process_tweets(TRAIN_PROCESSED_FILE, test_file=False)
    if TRAIN:
        train_tweets, val_tweets = split_data(tweets)
    else:
        random.shuffle(tweets)
        train_tweets = tweets
    del tweets
    print ('Extracting features & training batches')
    clf = svm.LinearSVC(C=0.1)
    batch_size = len(train_tweets)
    i = 1
    n_train_batches = int(np.ceil(len(train_tweets) / float(batch_size)))
    for training_set_X, training_set_y in extract_features(train_tweets, test_file=False, feat_type=FEAT_TYPE, batch_size=batch_size):
        write_status(i, n_train_batches)
        i += 1
        if FEAT_TYPE == 'frequency':
            tfidf = apply_tf_idf(training_set_X)
            training_set_X = tfidf.transform(training_set_X)
        clf.fit(training_set_X, training_set_y)
    print ('\n')
    print ('Testing')
    if TRAIN:
        correct, total = 0, len(val_tweets)
        i = 1
        batch_size = len(val_tweets)
        n_val_batches = int(np.ceil(len(val_tweets) / float(batch_size)))
        for val_set_X, val_set_y in extract_features(val_tweets, test_file=False, feat_type=FEAT_TYPE, batch_size=batch_size):
            if FEAT_TYPE == 'frequency':
                val_set_X = tfidf.transform(val_set_X)
            prediction = clf.predict(val_set_X)
            correct += np.sum(prediction == val_set_y)
            write_status(i, n_val_batches)
            i += 1
        print ('\nCorrect: %d/%d = %.4f %%' % (correct, total, correct * 100. / total))
    else:
        del train_tweets
        test_tweets = process_tweets(TEST_PROCESSED_FILE, test_file=True)
        n_test_batches = int(np.ceil(len(test_tweets) / float(batch_size)))
        predictions = np.array([])
        print ('Predicting batches')
        i = 1
        for test_set_X, _ in extract_features(test_tweets, test_file=True, feat_type=FEAT_TYPE):
            if FEAT_TYPE == 'frequency':
                test_set_X = tfidf.transform(test_set_X)
            prediction = clf.predict(test_set_X)
            predictions = np.concatenate((predictions, prediction))
            write_status(i, n_test_batches)
            i += 1
        predictions = [(str(j), int(predictions[j]))
                       for j in range(len(test_tweets))]
        save_results_to_csv(predictions, '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/svm.csv')
        print ('\nSaved to svm.csv')

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1

Testing
Processing 1/1
Correct: 7845/10000 = 78.4500 %


In [ ]:
!python2 /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/code/svm.py

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1

Testing
Processing 1/1
Correct: 7892/10000 = 78.9200 %


# **Multi-Layer Perceptron**

neuralnet.py

In [5]:
from keras.models import Sequential, load_model
from keras.layers import Dense
import sys #mengakses konfigurasi interpreter
import random #untuk  mengubah urutan setiap elemen yang ada pada list
import numpy as np #untuk melakukan komputasi numerik

# Performs classification using an MLP/1-hidden-layer NN.

FREQ_DIST_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist.pkl'
BI_FREQ_DIST_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist-bi.pkl'
TRAIN_PROCESSED_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed.csv'
TEST_PROCESSED_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/test-processed.csv'
TRAIN = True
UNIGRAM_SIZE = 15000
VOCAB_SIZE = UNIGRAM_SIZE
USE_BIGRAMS = False
if USE_BIGRAMS:
    BIGRAM_SIZE = 10000
    VOCAB_SIZE = UNIGRAM_SIZE + BIGRAM_SIZE
FEAT_TYPE = 'frequency'
xrange = range

def get_feature_vector(tweet): #untuk mendapatkan vektor pada tweet
    uni_feature_vector = []
    bi_feature_vector = []
    words = tweet.split()
    for i in xrange(len(words) - 1):
        word = words[i]
        next_word = words[i + 1]
        if unigrams.get(word):
            uni_feature_vector.append(word)
        if USE_BIGRAMS:
            if bigrams.get((word, next_word)):
                bi_feature_vector.append((word, next_word))
    if len(words) >= 1:
        if unigrams.get(words[-1]):
            uni_feature_vector.append(words[-1])
    return uni_feature_vector, bi_feature_vector


def extract_features(tweets, batch_size=500, test_file=True, feat_type='presence'): #melakukan ekstrasi feature tweet
    num_batches = int(np.ceil(len(tweets) / float(batch_size)))
    for i in xrange(num_batches):
        batch = tweets[i * batch_size: (i + 1) * batch_size]
        features = np.zeros((batch_size, VOCAB_SIZE))
        labels = np.zeros(batch_size)
        for j, tweet in enumerate(batch):
            if test_file:
                tweet_words = tweet[1][0]
                tweet_bigrams = tweet[1][1]
            else:
                tweet_words = tweet[2][0]
                tweet_bigrams = tweet[2][1]
                labels[j] = tweet[1]
            if feat_type == 'presence':
                tweet_words = set(tweet_words)
                tweet_bigrams = set(tweet_bigrams)
            for word in tweet_words:
                idx = unigrams.get(word)
                if idx:
                    features[j, idx] += 1
            if USE_BIGRAMS:
                for bigram in tweet_bigrams:
                    idx = bigrams.get(bigram)
                    if idx:
                        features[j, UNIGRAM_SIZE + idx] += 1
        yield features, labels


def process_tweets(csv_file, test_file=True):
    tweets = []
    print ('Generating feature vectors')
    with open(csv_file, 'r') as csv:
        lines = csv.readlines()
        total = len(lines)
        for i, line in enumerate(lines):
            if test_file:
                tweet_id, tweet = line.split(',')
            else:
                tweet_id, sentiment, tweet = line.split(',')
            feature_vector = get_feature_vector(tweet)
            if test_file:
                tweets.append((tweet_id, feature_vector))
            else:
                tweets.append((tweet_id, int(sentiment), feature_vector))
            write_status(i + 1, total)
    print ('\n')
    return tweets


def build_model():
    model = Sequential()
    model.add(Dense(500, input_dim=VOCAB_SIZE, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    return model


def evaluate_model(model, val_tweets):
    correct, total = 0, len(val_tweets)
    for val_set_X, val_set_y in extract_features(val_tweets, feat_type=FEAT_TYPE, test_file=False):
        prediction = model.predict_on_batch(val_set_X)
        prediction = np.round(prediction)
        correct += np.sum(prediction == val_set_y[:, None])
    return float(correct) / total


if __name__ == '__main__':
    np.random.seed(1337)
    unigrams = top_n_words(FREQ_DIST_FILE, UNIGRAM_SIZE)
    if USE_BIGRAMS:
        bigrams = top_n_bigrams(BI_FREQ_DIST_FILE, BIGRAM_SIZE)
    tweets = process_tweets(TRAIN_PROCESSED_FILE, test_file=False)
    if TRAIN:
        train_tweets, val_tweets = split_data(tweets)
    else:
        random.shuffle(tweets)
        train_tweets = tweets
    del tweets
    print ('Extracting features & training batches')
    nb_epochs = 5
    batch_size = 500
    model = build_model()
    n_train_batches = int(np.ceil(len(train_tweets) / float(batch_size)))
    best_val_acc = 0.0
    for j in xrange(nb_epochs):
        i = 1
        for training_set_X, training_set_y in extract_features(train_tweets, feat_type=FEAT_TYPE, batch_size=batch_size, test_file=False):
            o = model.train_on_batch(training_set_X, training_set_y)
            sys.stdout.write('\rIteration %d/%d, loss:%.4f, acc:%.4f' %
                             (i, n_train_batches, o[0], o[1]))
            sys.stdout.flush()
            i += 1
        val_acc = evaluate_model(model, val_tweets)
        print ('\nEpoch: %d, val_acc:%.4f' % (j + 1, val_acc))
        random.shuffle(train_tweets)
        if val_acc > best_val_acc:
            print ('Accuracy improved from %.4f to %.4f, saving model' % (best_val_acc, val_acc))
            best_val_acc = val_acc
            model.save('best_model.h5')
    print ('Testing')
    del train_tweets
    del model
    model = load_model('best_model.h5')
    test_tweets = process_tweets(TEST_PROCESSED_FILE, test_file=True)
    n_test_batches = int(np.ceil(len(test_tweets) / float(batch_size)))
    predictions = np.array([])
    print ('Predicting batches')
    i = 1
    for test_set_X, _ in extract_features(test_tweets, feat_type=FEAT_TYPE, batch_size=batch_size, test_file=True):
        prediction = np.round(model.predict_on_batch(test_set_X).flatten())
        predictions = np.concatenate((predictions, prediction))
        write_status(i, n_test_batches)
        i += 1
    predictions = [(str(j), int(predictions[j]))
                   for j in range(len(test_tweets))]
    save_results_to_csv(predictions, '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/1layerneuralnet.csv')
    print ('\nSaved to 1layerneuralnet.csv')

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Iteration 180/180, loss:0.5645, acc:0.7080
Epoch: 1, val_acc:0.7537
Accuracy improved from 0.0000 to 0.7537, saving model
Iteration 180/180, loss:0.4692, acc:0.7760
Epoch: 2, val_acc:0.7621
Accuracy improved from 0.7537 to 0.7621, saving model
Iteration 180/180, loss:0.4520, acc:0.8180
Epoch: 3, val_acc:0.7619
Iteration 180/180, loss:0.4088, acc:0.8260
Epoch: 4, val_acc:0.7570
Iteration 180/180, loss:0.4361, acc:0.7880
Epoch: 5, val_acc:0.7557
Testing
Generating feature vectors
Processing 300000/300000

Predicting batches
Processing 600/600
Saved to 1layerneuralnet.csv


In [3]:
!python3 /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/code/neuralnet.py

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
2022-01-24 09:02:09.334257: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Iteration 180/180, loss:0.4841, acc:0.7860
Epoch: 1, val_acc:0.7613
Accuracy improved from 0.0000 to 0.7613, saving model
Iteration 180/180, loss:0.4915, acc:0.7680
Epoch: 2, val_acc:0.7681
Accuracy improved from 0.7613 to 0.7681, saving model
Iteration 180/180, loss:0.4521, acc:0.7960
Epoch: 3, val_acc:0.7628
Iteration 180/180, loss:0.4391, acc:0.8020
Epoch: 4, val_acc:0.7655
Iteration 180/180, loss:0.4161, acc:0.7920
Epoch: 5, val_acc:0.7649
Testing
Generating feature vectors
Processing 300000/300000

Predicting batches
Processing 600/600
Saved to 1layerneuralnet.csv
